In [158]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from scipy.stats import pearsonr as pearsonr
from scipy.stats import skew as skew
import math



In [159]:
def y_Ao(annotations):
    total = 0.0
    pairings = [x for x in itertools.combinations(annotations,2)]
    for a1,a2 in pairings:
        total+=int(a1==a2)
    return total / len(pairings)


def cleanstring(s):
    if type(s) == float or type(s) == int:
        if math.isnan(s):
            return ""
    if not s:
        return ""
    s = str(s).strip()
    s = s.replace("&#44",", ")
    if not s:
        return ""
    else:
        return s

class AnnotatedInstance:
    def __init__(self):
        self.leftcontext = ''
        self.rightcontext = ''
        self.headword = ''
        self.labels = []
        self.times = []
        
    def length(self):
        return len((str(self.leftcontext)+" "+str(self.rightcontext)).split())
    
    def Ao(self):
        return y_Ao(self.labels)
    
    def avg_time(self,threshold=100):
        return [x for x in self.times if x < threshold]
    
    def _normlabels(self):
        d = {}
        d['ANIM']='LIT'
        d['Category1']='LIT'
        
        d['MEAT']='MET'
        d['Categpory2']='MET'
        d['Category2']='MET'
        
        d['DOT']='DOT'
        d['Category4']='DOT'
        d['Category3']='DOT'
        return [d[x] for x in self.labels]
    
    def label(self):
        #LIT, MET, DOT
        normlabels = self._normlabels()
        if normlabels.count('LIT') > normlabels.count('MET') and normlabels.count('DOT') <= normlabels.count('LIT'):
            return 'LIT'
        if normlabels.count('MET') > normlabels.count('LIT') and normlabels.count('DOT') <= normlabels.count('MET'):
            return 'MET'
        else:
            return 'DOT'
        
    def __str__(self):
        return '-'.join([i for i in [self.leftcontext,self.headword,self.rightcontext]+self.labels+[str(x) for x in self.times]])
    
    
def CreateInstanceDict(dataframe):
    instancedict = {}
    for hitid in set(dataframe.Input_globalindex):
        inst = AnnotatedInstance()
        inst.times = list(dataframe[dataframe.Input_globalindex==hitid].WorkTimeInSeconds)
        inst.labels = list(dataframe[dataframe.Input_globalindex==hitid].Answer_Category)
        inst.leftcontext = cleanstring(list(dataframe[dataframe.Input_globalindex==hitid].Input_lleftcontext)[0])
        inst.rightcontext = cleanstring(list(dataframe[dataframe.Input_globalindex==hitid].Input_lrightcontext)[0])
        inst.headword = cleanstring(list(dataframe[dataframe.Input_globalindex==hitid].Input_lheadword)[0])
        instancedict[hitid]=inst
    return instancedict
    
def CreateInstanceDict2(dataframe):
    instancedict = {}
    for hitid in set(dataframe.HITId):
        inst = AnnotatedInstance()
        inst.times = list(dataframe[dataframe.HITId==hitid].WorkTimeInSeconds)
        inst.labels = list(dataframe[dataframe.HITId==hitid].Answer_Category)
        inst.leftcontext = cleanstring(list(dataframe[dataframe.HITId==hitid].Input_lleftcontext)[0])
        inst.rightcontext = cleanstring(list(dataframe[dataframe.HITId==hitid].Input_lrightcontext)[0])
        inst.headword = cleanstring(list(dataframe[dataframe.HITId==hitid].Input_lheadword)[0])
        instancedict[hitid]=inst
    return instancedict
        

def Datasettime(instancedict):
    t =[]
    for k in instancedict:
        t.extend(instancedict[k].times)
    return t
    
def ReportDataset(instancedict):
    alltimes = Datasettime(instancedict)
    percentile95 =  np.percentile(alltimes, 95) #right-hand outliers filtered, remove anything above the 95th percentile
    alltimes= [x for x in alltimes if x < percentile95]
    
    mean = np.mean(alltimes)
    median = np.median(alltimes)
    std = np.std(alltimes)
    Ao_indiv_list = [instancedict[k].Ao() for k in instancedict.keys()]
    Ao_avg = np.mean(Ao_indiv_list)
    time_indiv_list =  [np.mean(instancedict[k].avg_time(percentile95)) for k in instancedict.keys()]
    lenghts = [instancedict[k].length() for k in instancedict.keys()]
    labels = [instancedict[k].label() for k in instancedict.keys()]
    #return (mean, median, std,percentile95,Ao_avg,pearsonr(Ao_indiv_list,time_indiv_list),pearsonr(lenghts,time_indiv_list)[0],Counter(labels)['LIT'])
    return (pearsonr(Ao_indiv_list,time_indiv_list)[0],pearsonr(lenghts,time_indiv_list)[0],Counter(labels)['LIT'])


def getLiteral(inputlist):
    litlabels = ['Category1', 'ANIM']
    return [int(x in litlabels) for x in inputlist]



In [160]:
animeat = pd.read_csv('data/raw/animeatbatchresults.csv')
contcont = pd.read_csv('data/raw/container_5turks.tab.csv',sep='\t')
artinfo = pd.read_csv('data/raw/artinfo_mturk_500.tab.csv',sep='\t')
procres = pd.read_csv('data/raw/procres_mturk.tab.csv',sep='\t')
locorg = pd.read_csv('data/raw/locorg_5turks.tab.csv',sep='\t')

In [161]:
instancedict = CreateInstanceDict(locorg)
print(ReportDataset(instancedict))

for k in instancedict.keys():
    try:
        str(instancedict[k])
    except:
        print("error error error",k)



(-0.060481137096010773, 0.063840031407057574, 313)


In [162]:
datataframes = [animeat, artinfo, contcont, locorg, procres]
names = ['animeat', 'artinfo', 'contcont', 'locorg','procres']
instancedicts = {}
for dataframe,name in zip(datataframes,names):
    instancedicts[name] = CreateInstanceDict(dataframe)
    
for k in sorted(instancedicts.keys()):
    print(k,ReportDataset(instancedicts[k]))


animeat (-0.37962472276825499, 0.30349345562040353, 362)
artinfo (-0.077477910735849531, 0.3961781984082135, 147)
contcont (-0.074817980159857786, 0.052722076290081826, 361)
locorg (-0.060481137096010773, 0.063840031407057574, 313)
procres (-0.073207870672663863, 0.10057235289900356, 158)


In [163]:
"From here we can see that all datasets have a different ... AND very different mean annotation time."


'From here we can see that all datasets have a different ... AND very different mean annotation time.'

In [164]:
for x in [animeat, artinfo, contcont, locorg,procres]:
    lit = getLiteral()
    print(pearsonr(lit,np.array(x.WorkTimeInSeconds)))

TypeError: getLiteral() missing 1 required positional argument: 'inputlist'